In [1]:
import sys
import json
import yaml
import inspect
from enum import Enum

import torch
# torch.set_printoptions(profile="full")

In [2]:
# this is based off the docs here and the file native_functions.yaml in the pytorch github repo
# https://github.com/pytorch/pytorch/tree/master/aten/src/ATen/native

In [3]:
!curl -O https://raw.githubusercontent.com/pytorch/pytorch/1.6/aten/src/ATen/native/native_functions.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  267k  100  267k    0     0   492k      0 --:--:-- --:--:-- --:--:--  491k


In [4]:
with open(r"native_functions.yaml") as file:
    native_functions = yaml.load(file, Loader=yaml.FullLoader)

In [5]:
# example structure
# - func: add_relu.out(Tensor self, Tensor other, *, Scalar alpha=1, Tensor(a!) out) -> Tensor(a!)
#   variants: function
#   dispatch:
#     CPU: add_relu_out

def process_yaml_entry(native_func):
    return_type = native_func["func"].split("->")[-1].strip()

    func_name = native_func["func"].split("(")[0].strip()
    args_parts = native_func["func"].split("->")[0].strip().split("(")
    args = "".join(args_parts[1:]).strip().replace("(", "").replace(")", "")
    return func_name, args, return_type

print(native_functions[0])
print(process_yaml_entry(native_functions[0]))

{'func': '_cast_Byte(Tensor self, bool non_blocking=False) -> Tensor', 'use_c10_dispatcher': 'full', 'variants': 'function'}
('_cast_Byte', 'Tensor self, bool non_blocking=False', 'Tensor')


In [6]:
def return_type_to_python(return_type):
    return_type = return_type.strip()
    if "()" == return_type:
        return ["None"]

    return_type = return_type.replace("(", "").replace(")", "")
    parts = return_type.split(",")
    clean_parts = []
    for part in parts:
        clean_part = part.strip()
        if clean_part.startswith("Tensor"):
            clean_part = "Tensor"
        # according to the docs Scalar is any kind of numeric in python or a unit tensor
        if clean_part == "Scalar":
            clean_part = "Union[int, float, complex, Tensor]"
        # it seems like these are the pytorch dtype types
        if clean_part == "ScalarType":
            clean_part = "torch.dtype"
        if "[]" in part:
            clean_part = f"List[{clean_part}]"
        
        clean_parts.append(clean_part)
    return clean_parts
    
    if any(prim == return_type for prim in ["int", "float", "bool"]):
        return return_type
    
return_type_to_python("(Tensor(a!) eigenvalues, Tensor(b!) eigenvectors)")

['Tensor', 'Tensor']

In [7]:
def signature_type_to_python(signature):
    signature = signature.strip()

    signature = signature.replace("(", "").replace(")", "")
    clean_parts = []

    if signature == "":
        return ["None"]
    # default int[] value [0,1] should not add space after comma, since native_parse.py uses ', ' to split args
    for part in signature.split(", "):
        name = None
        default = None
        clean_part = part.strip()
        if clean_part == "*":
            # all args after this must be kwargs
            # not sure we care right now
#             clean_parts.append(clean_part)
            continue
        subpart = clean_part.split(" ")
        if len(subpart) > 1:
            subsub = subpart[1].split("=")
            name = subsub[0]
            if len(subsub) > 1:
                default = subsub[1]

        if subpart[0].startswith("Tensor"):
            clean_part = "Tensor"
        else:
            clean_part = subpart[0]

        if clean_part.startswith("ScalarType"):
            clean_part = "torch.dtype"
            
        if clean_part.startswith("Scalar"):
            clean_part = "Union[int, float, complex, Tensor]"

        if "[" in subpart[0]:
            t = clean_part.split("[")[0]
            clean_part = f"List[{t}]"

        if "?" in part:
            clean_part = f"Optional[{clean_part}]".replace("?", "")
            
        if name is not None:
            clean_part = f"{name}: {clean_part}"
            
        if default is not None:
            clean_part = f"{clean_part} = {default}"

        clean_parts.append(clean_part)
        
    if len(clean_parts) == 0:
        return ["None"]
    return clean_parts

signature_type_to_python( 'Scalar end, *, ScalarType? dtype=None, Layout? layout=None, Device? device=None, bool? pin_memory=None')

['end: Union[int, float, complex, Tensor]',
 'dtype: Optional[torch.dtype] = None',
 'layout: Optional[Layout] = None',
 'device: Optional[Device] = None',
 'pin_memory: Optional[bool] = None']

In [8]:
def process_func_yaml(native_functions):
    unique_return_types = set()
    unique_function_args = set()
    func_dict = {}
    for native_func in native_functions:
        func_name, args, return_type = process_yaml_entry(native_func)
        unique_return_types.add(return_type)
        unique_function_args.add(args)

        if func_name in func_dict:
            print(f"Error duplicate func_name: {func_name}")
        func_dict[func_name] = {"args": signature_type_to_python(args), "return_type": return_type_to_python(return_type)}

    return func_dict, unique_return_types, unique_function_args

In [9]:
native_func_dict, unique_return_types, unique_function_args = process_func_yaml(native_functions)
print(len(native_func_dict))

1527


In [10]:
print(native_func_dict["div_.Tensor"])

type_json = json.dumps(native_func_dict)
with open("torch_types.json", "w") as f:
    f.write(type_json)

{'args': ['self: Tensor', 'other: Tensor'], 'return_type': ['Tensor']}


In [11]:
unique_return_types

{'()',
 '(Tensor Q, Tensor R)',
 '(Tensor U, Tensor S, Tensor V)',
 '(Tensor a, Tensor tau)',
 '(Tensor eigenvalues, Tensor eigenvectors)',
 '(Tensor grad_input, Tensor grad_weight)',
 '(Tensor grad_input, Tensor grad_weight, Tensor grad_bias)',
 '(Tensor grad_self, Tensor grad_grid)',
 '(Tensor output, Tensor buffer)',
 '(Tensor output, Tensor finput, Tensor fgrad_input)',
 '(Tensor output, Tensor is_target)',
 '(Tensor output, Tensor total_weight)',
 '(Tensor sign, Tensor logabsdet)',
 '(Tensor solution, Tensor LU)',
 '(Tensor solution, Tensor QR)',
 '(Tensor solution, Tensor cloned_coefficient)',
 '(Tensor values, Tensor indices)',
 '(Tensor(a!) Q, Tensor(b!) R)',
 '(Tensor(a!) U, Tensor(b!) S, Tensor(c!) V)',
 '(Tensor(a!) a, Tensor(b!) tau)',
 '(Tensor(a!) eigenvalues, Tensor(b!) eigenvectors)',
 '(Tensor(a!) solution, Tensor(b!) LU)',
 '(Tensor(a!) solution, Tensor(b!) QR)',
 '(Tensor(a!) solution, Tensor(b!) cloned_coefficient)',
 '(Tensor(a!) values, Tensor(b!) indices)',
 '(Te

In [12]:
unique_python_return_types = set()
for line in [return_type_to_python(t) for t in unique_return_types]:
    unique_python_return_types.add(tuple(line))

unique_python_return_types

{('List[Tensor]',),
 ('None',),
 ('QScheme',),
 ('Tensor',),
 ('Tensor', 'Tensor'),
 ('Tensor', 'Tensor', 'Tensor'),
 ('Tensor', 'Tensor', 'Tensor', 'List[Tensor]'),
 ('Tensor', 'Tensor', 'Tensor', 'Tensor'),
 ('Tensor', 'Tensor', 'Tensor', 'Tensor', 'Tensor'),
 ('Tensor', 'Tensor', 'Tensor', 'Tensor', 'int'),
 ('Tensor', 'Tensor', 'float', 'int'),
 ('Union[int, float, complex, Tensor]',),
 ('bool',),
 ('float',),
 ('float', 'int'),
 ('int',),
 ('torch.dtype',)}

In [13]:
list(unique_function_args)[0:10]

['',
 'Tensor self, Tensor mat, Tensor vec, *, Scalar beta=1, Scalar alpha=1, Tensora! out',
 'int sparse_dim, int dense_dim, int[] size, *, ScalarType? dtype=None, Layout? layout=None, Device? device=None, bool? pin_memory=False',
 'Tensor self, int[2] padding, *, Tensora! out',
 'int n, Tensor self, *, Tensora! out',
 'Tensor self, Tensor vec1, Tensor vec2, *, Scalar beta=1, Scalar alpha=1',
 'Tensor grad_output, Tensor self, Tensor target, Tensor? weight=None, int reduction=Mean',
 'Tensor self, int dim, Tensor index',
 'Tensor self, Tensor weight, Tensor? bias, int[] padding, int[] output_padding, int[] stride, int[] dilation, int groups, bool benchmark, bool deterministic',
 'Tensor grad_output, int[2] output_size, int[4] input_size, float? scales_h=None, float? scales_w=None']

In [14]:
unique_python_arg_signatures = set()
for line in [signature_type_to_python(t) for t in unique_function_args]:
    unique_python_arg_signatures.add(tuple(line))
    
print(len(unique_python_arg_signatures))

list(unique_python_arg_signatures)[0:10]

716


[('self: Tensor', 'repeats: List[int]'),
 ('self: Tensor',
  'kernel_size: List[int]',
  'stride: List[int] = []',
  'padding: List[int] = 0',
  'dilation: List[int] = 1',
  'ceil_mode: bool = False'),
 ('self: Tensor', 'scale: Tensor', 'zero_point: Tensor', 'axis: int'),
 ('self: Tensor',
  'batch1: Tensor',
  'batch2: Tensor',
  'beta: Union[int, float, complex, Tensor] = 1',
  'alpha: Union[int, float, complex, Tensor] = 1'),
 ('self: Tensor',
  'dim: List[Dimname]',
  'unbiased: bool = True',
  'keepdim: bool = False'),
 ('low: int',
  'high: int',
  'size: List[int]',
  'generator: Optional[Generator]',
  'out: Tensor'),
 ('self: Tensor',
  'output_size: List[int]',
  'align_corners: bool',
  'scales_h: Optional[float] = None',
  'scales_w: Optional[float] = None'),
 ('self: Tensor', 'exponent: Union[int, float, complex, Tensor]'),
 ('start: Union[int, float, complex, Tensor]',
  'end: Union[int, float, complex, Tensor]',
  'steps: int = 100',
  'out: Tensor'),
 ('self: Tensor', '

In [15]:
arg_signatures_no_names = set()
for sig in unique_python_arg_signatures:
    new_parts = []
    for part in sig:
        new_part = part.split("=")[0]
        new_part = new_part.split(":")[-1]
        new_part = new_part.strip()
        new_parts.append(new_part)
    arg_signatures_no_names.add(tuple(new_parts))
    
print(len(arg_signatures_no_names))

list(arg_signatures_no_names)[0:10]

452


[('Tensor', 'Tensor', 'List[int]', 'List[int]', 'int', 'int', 'bool'),
 ('Tensor',
  'Tensor',
  'List[Tensor]',
  'bool',
  'int',
  'float',
  'bool',
  'bool',
  'bool'),
 ('Tensor', 'int', 'List[int]'),
 ('Tensor',
  'Tensor',
  'Tensor',
  'Optional[Tensor]',
  'Optional[Tensor]',
  'float',
  'float',
  'Tensor'),
 ('Union[int, float, complex, Tensor]',
  'Union[int, float, complex, Tensor]',
  'int',
  'float',
  'Optional[torch.dtype]',
  'Optional[Layout]',
  'Optional[Device]',
  'Optional[bool]'),
 ('Tensor', 'List[int]', 'bool', 'Optional[float]'),
 ('Tensor', 'ConstQuantizerPtr'),
 ('Tensor', 'Tensor', 'Tensor', 'int'),
 ('List[int]',
  'Optional[torch.dtype]',
  'Optional[Layout]',
  'Optional[Device]',
  'Optional[bool]'),
 ('Tensor', 'bool', 'bool', 'Tensor', 'Tensor')]

In [16]:
all_types = set()
for sigs in arg_signatures_no_names:
    for part in sigs:
        all_types.add(part)
        
for return_types in unique_python_return_types:
    for part in return_types:
        all_types.add(part)
        
print(len(all_types))
all_types

32


{'ConstQuantizerPtr',
 'Device',
 'Dimname',
 'List[Dimname]',
 'List[Tensor]',
 'List[bool]',
 'List[int]',
 'MemoryFormat',
 'None',
 'Optional[Device]',
 'Optional[Generator]',
 'Optional[Layout]',
 'Optional[List[Dimname]]',
 'Optional[List[Tensor]]',
 'Optional[List[float]]',
 'Optional[List[int]]',
 'Optional[MemoryFormat]',
 'Optional[Tensor]',
 'Optional[Union[int, float, complex, Tensor]]',
 'Optional[bool]',
 'Optional[float]',
 'Optional[int]',
 'Optional[torch.dtype]',
 'QScheme',
 'Storage',
 'Tensor',
 'Union[int, float, complex, Tensor]',
 'bool',
 'float',
 'int',
 'str',
 'torch.dtype'}

In [17]:
def full_name_with_qualname(klass: type) -> str:
    return f'{klass.__module__}.{klass.__qualname__}'

In [18]:
def whatis(thing):
    name = getattr(thing, "__name__", "")
    print(f"{thing} {type(thing)} {name}")
    
    mro = getattr(thing, "mro", None)
    if mro is not None:
        hierarchy = mro()
        print(f"Hierarchy: {hierarchy}")
    

    test_types = [
        "ismodule", "isclass", "ismethod", "ismethoddescriptor", "isfunction", "isgeneratorfunction", "isgenerator",
        "isbuiltin", "isroutine", "isdatadescriptor", "isgetsetdescriptor", "ismemberdescriptor"
    ]
    result_ttypes = []
    for ttype in test_types:
        istype = getattr(inspect, ttype)(thing)
        if istype is True:
            result_ttypes.append(ttype)

    if callable(thing):
        result_ttypes.append("callable")
       
    if issubclass(type(thing), Enum):
        result_ttype.append("ispyenum")

    members = getattr(thing, "__members__", None)
    if members is not None:
        print("members: ", members)
        result_ttypes.append("iscenum")
    
    if "pybind11_type" in str(type(thing)):
        result_ttypes.append("ispybind11")

    return result_ttypes

In [19]:
def get_signature(thing):
    return inspect.getfullargspec(thing.__init__)

In [20]:
def get_c_class_signature(thing):
    signature = ""
    try:
        thing()
        signature = f"1. {thing}()"
    except Exception as e:
        if "No constructor defined" in str(e):
            return None

        for line in str(e).splitlines():
            if any(parts in line.strip() for parts in ["TypeError", "incompatible", "Invoked with:"]):
                continue
            else:
                signature += f"{line}\n"
        signature = signature.strip()

    return signature

In [21]:
def get_native_signature(key):
    if key in native_func_dict:
        # print(f"Found {key} in native_func_dict")
        return native_func_dict[key]["args"]
    else:
        print(f"Cant find {key} in native_func_dict")
        return None

In [22]:
def get_native_return_type(key):
    if key in native_func_dict:
        # print(f"Found {key} in native_func_dict")
        return native_func_dict[key]["return_type"]
    else:
        print(f"Cant find {key} in native_func_dict")
        return None
    

In [23]:
missing_attr_matches = []
def detect_attrs(obj) -> dict:
    path_dict = {}
    attr_dict = {"module":{}, "enum": {}, "class": {}, "method":{}, "function":{}, "property":{}, "builtin": {}}
    
    count = 0
    for key in dir(obj):
        try:
            if key.startswith("__"):
                continue
            print("checking", key)
            count += 1
    #         if count > 20:
    #             break
            attr = getattr(obj, key)
            prop_types = whatis(attr)
            print(key, prop_types)
            if "iscenum" in prop_types and "ispybind11" not in prop_types:
                attr_dict["enum"][type(attr)] = {"members":type(attr).__members__}
                path_dict[key] = {"type": "enum", "members":type(attr).__members__}
            if "isclass" in prop_types and "ispybind11" not in prop_types:
                attr_dict["class"][key] = get_signature(attr)
                path_dict[key] = {"type": "class", "init":get_signature(attr)}
            if "isclass" in prop_types and "ispybind11" in prop_types:
                attr_dict["class"][key] = get_c_class_signature(attr)
                path_dict[key] = {"type": "class", "init":get_c_class_signature(attr)}

            if "isbuiltin" in prop_types and "isroutine" in prop_types:
                if get_native_signature(key) is None or get_native_return_type(key) is None:
                    missing_attr_matches.append(key)
                path_dict[key] = {
                    "type": "function",
                    "signature": get_native_signature(key),
                    "return_type": get_native_return_type(key)
                }

        except Exception as e:
            print(f"Exception with {key}. {e}")
    return path_dict

attrs = detect_attrs(torch)

checking AVG
AggregationType.AVG <class 'torch._C.AggregationType'> 
members:  {'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}
AVG ['iscenum']
checking AggregationType
<class 'torch._C.AggregationType'> <class 'pybind11_builtins.pybind11_type'> AggregationType
Hierarchy: [<class 'torch._C.AggregationType'>, <class 'pybind11_builtins.pybind11_object'>, <class 'object'>]
members:  {'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}
AggregationType ['isclass', 'callable', 'iscenum', 'ispybind11']
checking AnyType
<class 'torch._C.AnyType'> <class 'pybind11_builtins.pybind11_type'> AnyType
Hierarchy: [<class 'torch._C.AnyType'>, <class 'torch._C.Type'>, <class 'pybind11_builtins.pybind11_object'>, <class 'object'>]
AnyType ['isclass', 'callable', 'ispybind11']
checking Argument
<class 'torch._C.Argument'> <class 'pybind11_builtins.pybind11_type'> Argument
Hierarchy: [<class 'torch._C.Argument'>, <class 'pybind11_builtins.pybind11_object'>, <class 'object'>]
Argument ['isclass

False <class 'bool'> 
USE_RTLD_GLOBAL_WITH_LIBTORCH []
checking Use
<class 'torch._C.Use'> <class 'pybind11_builtins.pybind11_type'> Use
Hierarchy: [<class 'torch._C.Use'>, <class 'pybind11_builtins.pybind11_object'>, <class 'object'>]
Use ['isclass', 'callable', 'ispybind11']
checking Value
<class 'torch._C.Value'> <class 'pybind11_builtins.pybind11_type'> Value
Hierarchy: [<class 'torch._C.Value'>, <class 'pybind11_builtins.pybind11_object'>, <class 'object'>]
Value ['isclass', 'callable', 'ispybind11']
checking _C
<module 'torch._C' from '/Users/madhavajay/.local/share/virtualenvs/PySyft-lHlz_cKe/lib/python3.6/site-packages/torch/_C.cpython-36m-darwin.so'> <class 'module'> torch._C
_C ['ismodule']
checking _StorageBase
<class 'torch.storage._StorageBase'> <class 'type'> _StorageBase
Hierarchy: [<class 'torch.storage._StorageBase'>, <class 'object'>]
_StorageBase ['isclass', 'callable']
checking _VF
<module 'torch._VF'> <class 'torch._VF.VFModule'> torch._VF
_VF ['ismodule']
checking

addcdiv ['isbuiltin', 'isroutine', 'callable']
checking addcmul
<built-in method addcmul of type object at 0x1144465b0> <class 'builtin_function_or_method'> addcmul
addcmul ['isbuiltin', 'isroutine', 'callable']
checking addmm
<built-in method addmm of type object at 0x1144465b0> <class 'builtin_function_or_method'> addmm
addmm ['isbuiltin', 'isroutine', 'callable']
checking addmv
<built-in method addmv of type object at 0x1144465b0> <class 'builtin_function_or_method'> addmv
addmv ['isbuiltin', 'isroutine', 'callable']
checking addmv_
<built-in method addmv_ of type object at 0x1144465b0> <class 'builtin_function_or_method'> addmv_
addmv_ ['isbuiltin', 'isroutine', 'callable']
checking addr
<built-in method addr of type object at 0x1144465b0> <class 'builtin_function_or_method'> addr
addr ['isbuiltin', 'isroutine', 'callable']
checking affine_grid_generator
<built-in method affine_grid_generator of type object at 0x1144465b0> <class 'builtin_function_or_method'> affine_grid_generator


Cant find conv_transpose3d in native_func_dict
checking convolution
<built-in method convolution of type object at 0x1144465b0> <class 'builtin_function_or_method'> convolution
convolution ['isbuiltin', 'isroutine', 'callable']
checking cos
<built-in method cos of type object at 0x1144465b0> <class 'builtin_function_or_method'> cos
cos ['isbuiltin', 'isroutine', 'callable']
checking cos_
<built-in method cos_ of type object at 0x1144465b0> <class 'builtin_function_or_method'> cos_
cos_ ['isbuiltin', 'isroutine', 'callable']
checking cosh
<built-in method cosh of type object at 0x1144465b0> <class 'builtin_function_or_method'> cosh
cosh ['isbuiltin', 'isroutine', 'callable']
checking cosh_
<built-in method cosh_ of type object at 0x1144465b0> <class 'builtin_function_or_method'> cosh_
cosh_ ['isbuiltin', 'isroutine', 'callable']
checking cosine_embedding_loss
<built-in method cosine_embedding_loss of type object at 0x1144465b0> <class 'builtin_function_or_method'> cosine_embedding_loss


checking floor_divide
<built-in method floor_divide of type object at 0x1144465b0> <class 'builtin_function_or_method'> floor_divide
floor_divide ['isbuiltin', 'isroutine', 'callable']
checking fmod
<built-in method fmod of type object at 0x1144465b0> <class 'builtin_function_or_method'> fmod
fmod ['isbuiltin', 'isroutine', 'callable']
Cant find fmod in native_func_dict
Cant find fmod in native_func_dict
Cant find fmod in native_func_dict
checking fork
<built-in method fork of PyCapsule object at 0x114ca6030> <class 'builtin_function_or_method'> fork
fork ['isbuiltin', 'isroutine', 'callable']
Cant find fork in native_func_dict
Cant find fork in native_func_dict
Cant find fork in native_func_dict
checking frac
<built-in method frac of type object at 0x1144465b0> <class 'builtin_function_or_method'> frac
frac ['isbuiltin', 'isroutine', 'callable']
checking frac_
<built-in method frac_ of type object at 0x1144465b0> <class 'builtin_function_or_method'> frac_
frac_ ['isbuiltin', 'isroutin

le ['isbuiltin', 'isroutine', 'callable']
Cant find le in native_func_dict
Cant find le in native_func_dict
Cant find le in native_func_dict
checking legacy_contiguous_format
torch.contiguous_format <class 'torch.memory_format'> 
legacy_contiguous_format []
checking lerp
<built-in method lerp of type object at 0x1144465b0> <class 'builtin_function_or_method'> lerp
lerp ['isbuiltin', 'isroutine', 'callable']
Cant find lerp in native_func_dict
Cant find lerp in native_func_dict
Cant find lerp in native_func_dict
checking lgamma
<built-in method lgamma of type object at 0x1144465b0> <class 'builtin_function_or_method'> lgamma
lgamma ['isbuiltin', 'isroutine', 'callable']
checking linspace
<built-in method linspace of type object at 0x1144465b0> <class 'builtin_function_or_method'> linspace
linspace ['isbuiltin', 'isroutine', 'callable']
checking load
<function load at 0x115a861e0> <class 'function'> load
load ['isfunction', 'isroutine', 'callable']
checking lobpcg
<function lobpcg at 0x13

os ['ismodule']
checking pairwise_distance
<built-in method pairwise_distance of type object at 0x1144465b0> <class 'builtin_function_or_method'> pairwise_distance
pairwise_distance ['isbuiltin', 'isroutine', 'callable']
checking parse_ir
<built-in method parse_ir of PyCapsule object at 0x114ca3990> <class 'builtin_function_or_method'> parse_ir
parse_ir ['isbuiltin', 'isroutine', 'callable']
Cant find parse_ir in native_func_dict
Cant find parse_ir in native_func_dict
Cant find parse_ir in native_func_dict
checking parse_schema
<built-in method parse_schema of PyCapsule object at 0x114ca39c0> <class 'builtin_function_or_method'> parse_schema
parse_schema ['isbuiltin', 'isroutine', 'callable']
Cant find parse_schema in native_func_dict
Cant find parse_schema in native_func_dict
Cant find parse_schema in native_func_dict
checking parse_type_comment
<built-in method parse_type_comment of PyCapsule object at 0x114cc15d0> <class 'builtin_function_or_method'> parse_type_comment
parse_type_co

checking serialization
<module 'torch.serialization' from '/Users/madhavajay/.local/share/virtualenvs/PySyft-lHlz_cKe/lib/python3.6/site-packages/torch/serialization.py'> <class 'module'> torch.serialization
serialization ['ismodule']
checking set_anomaly_enabled
<built-in function set_anomaly_enabled> <class 'builtin_function_or_method'> set_anomaly_enabled
set_anomaly_enabled ['isbuiltin', 'isroutine', 'callable']
Cant find set_anomaly_enabled in native_func_dict
Cant find set_anomaly_enabled in native_func_dict
Cant find set_anomaly_enabled in native_func_dict
checking set_autocast_enabled
<built-in function set_autocast_enabled> <class 'builtin_function_or_method'> set_autocast_enabled
set_autocast_enabled ['isbuiltin', 'isroutine', 'callable']
Cant find set_autocast_enabled in native_func_dict
Cant find set_autocast_enabled in native_func_dict
Cant find set_autocast_enabled in native_func_dict
checking set_default_dtype
<function set_default_dtype at 0x115667e18> <class 'function'

zeros ['isbuiltin', 'isroutine', 'callable']
checking zeros_like
<built-in method zeros_like of type object at 0x1144465b0> <class 'builtin_function_or_method'> zeros_like
zeros_like ['isbuiltin', 'isroutine', 'callable']


In [24]:
missing_attr_matches

['add',
 'as_tensor',
 'autocast_decrement_nesting',
 'autocast_increment_nesting',
 'bitwise_and',
 'bitwise_or',
 'bitwise_xor',
 'bucketize',
 'clear_autocast_cache',
 'conv_transpose2d',
 'conv_transpose3d',
 'ctc_loss',
 'dequantize',
 'div',
 'dsmm',
 'empty',
 'eq',
 'fill_',
 'flatten',
 'fmod',
 'fork',
 'from_numpy',
 'ge',
 'get_default_dtype',
 'get_device',
 'get_num_interop_threads',
 'get_num_threads',
 'gru',
 'gt',
 'hsmm',
 'import_ir_module',
 'import_ir_module_from_buffer',
 'index_fill',
 'init_num_threads',
 'is_anomaly_enabled',
 'is_autocast_enabled',
 'is_grad_enabled',
 'le',
 'lerp',
 'log_softmax',
 'lstm',
 'lt',
 'masked_fill',
 'merge_type_from_type_comment',
 'mul',
 'ne',
 'normal',
 'numel',
 'parse_ir',
 'parse_schema',
 'parse_type_comment',
 'pow',
 'quantized_gru',
 'quantized_lstm',
 'remainder',
 'repeat_interleave',
 'result_type',
 'rnn_relu',
 'rnn_tanh',
 'rsub',
 'saddmm',
 'scatter',
 'searchsorted',
 'select',
 'set_anomaly_enabled',
 'set

In [25]:
for attr, meta in attrs.items():
    if meta["type"] == "enum":
        meta_info = meta["members"]
    if meta["type"] == "class":
        meta_info = meta["init"]
    if meta["type"] == "function":
        sig = meta["signature"]
        return_type = meta["return_type"]
        meta_info = f"{sig} => {return_type}"
    t = meta["type"]
    print(f"torch.{attr} - {t} ({meta_info})")

torch.AVG - enum ({'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG})
torch.AggregationType - class (1. torch._C.AggregationType(arg0: int))
torch.AnyType - class (None)
torch.Argument - class (None)
torch.ArgumentSpec - class (None)
torch.BFloat16Storage - class (FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))
torch.BFloat16Tensor - class (FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))
torch.BenchmarkConfig - class (1. <class 'torch._C.BenchmarkConfig'>())
torch.BenchmarkExecutionStats - class (None)
torch.Block - class (None)
torch.BoolStorage - class (FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))
torch.BoolTensor - class (FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations

torch.memory_format - class (FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))
torch.merge_type_from_type_comment - function (None => None)
torch.min - function (['self: Tensor'] => ['Tensor'])
torch.miopen_batch_norm - function (['input: Tensor', 'weight: Tensor', 'bias: Optional[Tensor]', 'running_mean: Optional[Tensor]', 'running_var: Optional[Tensor]', 'training: bool', 'exponential_average_factor: float', 'epsilon: float'] => ['Tensor', 'Tensor', 'Tensor'])
torch.miopen_convolution - function (['self: Tensor', 'weight: Tensor', 'bias: Optional[Tensor]', 'padding: List[int]', 'stride: List[int]', 'dilation: List[int]', 'groups: int', 'benchmark: bool', 'deterministic: bool'] => ['Tensor'])
torch.miopen_convolution_transpose - function (['self: Tensor', 'weight: Tensor', 'bias: Optional[Tensor]', 'padding: List[int]', 'output_padding: List[int]', 'stride: List[int]', 'dilation: List[int]', 'groups: int', 'b

In [26]:
# playground

In [27]:
issubclass(type(torch.AggregationType), Enum)
print(torch.AggregationType)
print(torch.AggregationType.SUM)

<class 'torch._C.AggregationType'>
AggregationType.SUM


In [28]:
print(torch.Tensor.set_.__doc__)
print(get_native_signature("set_"))
print(get_native_return_type("set_"))


set_(source=None, storage_offset=0, size=None, stride=None) -> Tensor

Sets the underlying storage, size, and strides. If :attr:`source` is a tensor,
:attr:`self` tensor will share the same storage and have the same size and
strides as :attr:`source`. Changes to elements in one tensor will be reflected
in the other.

If :attr:`source` is a :class:`~torch.Storage`, the method sets the underlying
storage, offset, size, and stride.

Args:
    source (Tensor or Storage): the tensor or storage to use
    storage_offset (int, optional): the offset in the storage
    size (torch.Size, optional): the desired size. Defaults to the size of the source.
    stride (tuple, optional): the desired stride. Defaults to C-contiguous strides.

['self: Tensor']
['Tensor']


In [29]:
whatis(torch.SUM)
torch.SUM.__members__

AggregationType.SUM <class 'torch._C.AggregationType'> 
members:  {'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}


{'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}

In [30]:
a = torch._C.AggregationType(0)
print(a)

AggregationType.SUM


In [31]:
print(torch.AVG.__members__)

{'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}


In [32]:
whatis(torch._C.AggregationType)

<class 'torch._C.AggregationType'> <class 'pybind11_builtins.pybind11_type'> AggregationType
Hierarchy: [<class 'torch._C.AggregationType'>, <class 'pybind11_builtins.pybind11_object'>, <class 'object'>]
members:  {'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}


['isclass', 'callable', 'iscenum', 'ispybind11']

In [33]:
print(torch.AVG.__members__)
# print(torch.AVG.mro())

#callable(torch.AVG)
#inspect.getmro(torch.AVG)
id(torch.AVG)
id(torch.SUM)

{'SUM': AggregationType.SUM, 'AVG': AggregationType.AVG}


4643865800

In [34]:
thing = torch.AVG
print(f"ismodule: {inspect.ismodule(thing)}")
print(f"isclass: {inspect.isclass(thing)}")
print(f"ismethod: {inspect.ismethod(thing)}")
print(f"ismethoddescriptor: {inspect.ismethoddescriptor(thing)}")
print(f"isfunction: {inspect.isfunction(thing)}")
print(f"isgeneratorfunction: {inspect.isgeneratorfunction(thing)}")
print(f"isgenerator: {inspect.isgenerator(thing)}")
print(f"isbuiltin: {inspect.isbuiltin(thing)}")
print(f"isroutine: {inspect.isroutine(thing)}")
print(f"isdatadescriptor: {inspect.isroutine(thing)}")
print(f"isgetsetdescriptor: {inspect.isroutine(thing)}")
print(f"ismemberdescriptor: {inspect.isroutine(thing)}")

ismodule: False
isclass: False
ismethod: False
ismethoddescriptor: False
isfunction: False
isgeneratorfunction: False
isgenerator: False
isbuiltin: False
isroutine: False
isdatadescriptor: False
isgetsetdescriptor: False
ismemberdescriptor: False


In [35]:
def compare_class_w_instance(klass, inst):
    a = set(list(dir(klass)))
    b = set(list(dir(inst)))

    print(a.difference(b))

In [36]:
compare_class_w_instance(torch.Tensor, torch.Tensor([1, 2, 3]))

{'volatile', '__cuda_array_interface__'}
